In [ ]:
from flask import Flask, render_template, request, jsonify
from flask_socketio import SocketIO
import numpy as np
import pickle


In [ ]:
# Import libraries for training
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Load the Iris dataset
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Save the trained model
with open("iris_model.pkl", "wb") as file:
    pickle.dump(model, file)

print("Model trained and saved successfully!")


In [ ]:
# Initialize Flask and Socket.IO
app = Flask(__name__)
socketio = SocketIO(app)

# Load the trained model
with open("iris_model.pkl", "rb") as file:
    model = pickle.load(file)

# Define the real-time prediction endpoint
@socketio.on('predict')
def handle_prediction(data):
    # Extract features from incoming data
    features = np.array(data['features']).reshape(1, -1)

    # Make prediction
    prediction = model.predict(features)
    predicted_class = iris.target_names[prediction[0]]

    # Send prediction back to the client
    response = {
        "input_features": data['features'],
        "predicted_class": predicted_class
    }
    socketio.emit('prediction_result', response)

# Define the home route
@app.route('/')
def home():
    return render_template('index.html')

# Run the app
if __name__ == '__main__':
    socketio.run(app, debug=True)


In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Real-Time Prediction</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.3.2/socket.io.js"></script>
</head>
<body>
    <h1>Real-Time Prediction System</h1>
    <form id="predictionForm">
        <label>Feature 1: <input type="number" id="feature1" step="any"></label><br>
        <label>Feature 2: <input type="number" id="feature2" step="any"></label><br>
        <label>Feature 3: <input type="number" id="feature3" step="any"></label><br>
        <label>Feature 4: <input type="number" id="feature4" step="any"></label><br>
        <button type="submit">Predict</button>
    </form>

    <h2>Prediction Result:</h2>
    <div id="result"></div>

    <script>
        const socket = io();

        document.getElementById('predictionForm').addEventListener('submit', function(event) {
            event.preventDefault();

            const features = [
                parseFloat(document.getElementById('feature1').value),
                parseFloat(document.getElementById('feature2').value),
                parseFloat(document.getElementById('feature3').value),
                parseFloat(document.getElementById('feature4').value)
            ];

            socket.emit('predict', { features: features });
        });

        socket.on('prediction_result', function(data) {
            document.getElementById('result').innerText = `Predicted Class: ${data.predicted_class}`;
        });
    </script>
</body>
</html>


In [ ]:
python app.py


In [ ]:
pip install gunicorn


In [ ]:
pip freeze > requirements.txt
echo "web: gunicorn app:app" > Procfile


In [ ]:
git init
git add .
git commit -m "Real-time prediction app"
heroku create
git push heroku master
